In [21]:
import sys
sys.path.insert(0,'../')
import os
#run on cpu to not interfere with trianing
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from net.hourglass import HourglassNet
from eval.heatmap_process import post_process_heatmap
import glob
import cv2
import matplotlib.pyplot as plt
import scipy
import imageio
from top.demo import render_joints
import numpy as np

In [22]:
num_class=14
num_stack=2
model_dir="../../trained_models/mpii_new"
model_json="net_arch.json"
epoch=49
model_weights="weights_epoch"+str(epoch)+".h5"
inres=(256, 256)
outres=(64,64)
xnet = HourglassNet(num_class, num_stack, inres, outres)
model_json=os.path.join(model_dir, model_json)
model_weights=os.path.join(model_dir, model_weights)

xnet.load_model(model_json, model_weights)


In [ ]:
img_dir="../../data/test_images"
# load files from data generator

for file in os.listdir(img_dir):
    print(file)
    img_file=os.path.join(img_dir,file)
    out,scale = xnet.inference_file(img_file)

    kps = post_process_heatmap(out[0,:,:,:])
    #first element of batch
    out=out[0]
    print(out.shape)
    for j in range(out.shape[-1]):
        print(out[:,:,j].shape)
        plt.imshow(out[:,:,j])
        plt.show()
    mkps = list() 
    for i, _kp in enumerate(kps):
        _conf = _kp[2]
        mkps.append((_kp[0]*scale[1]*4, _kp[1]*scale[0]*4, _conf))
    print(mkps)
    #normally conf th is around 0.2 but currently no joints are found
    cvmat = render_joints(imageio.imread(img_file), mkps, conf_th=0.002)
    plt.imshow(cvmat)
    plt.show()


000033016.jpg


/media/sda4/miniconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/media/sda4/miniconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:30: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
